In [1]:
#one prompt for all tipes of code
import tatqa_utils
import pandas as pd

In [2]:
devdf = pd.read_json('dataset_raw/tatqa_dataset_dev.json')

In [10]:
import pandas as pd
import pyreadstat
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
import utils
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
import numpy as np
from progress.bar import Bar
%load_ext autoreload
%autoreload 2

In [11]:
import os
with open('dataset_raw/openai.api.key', 'r') as filek: 
    openai_key = filek.read()
os.environ["OPENAI_API_KEY"] =  openai_key 

In [12]:
from langchain.globals import set_llm_cache
from langchain_openai import ChatOpenAI
from langchain_community.cache import SQLiteCache

llm = ChatOpenAI(temperature=0)
set_llm_cache(SQLiteCache(database_path=".langchain.db"))

In [13]:
def gen_code(llm, question, table):
    prompt = f"Given the following table, can you generate a python code, without sample data, which can answer the following question? the code must contain only one function called 'run', and no wrapping class. The function would return results with format (value, scale). Do not write explanation, just code.\nQuestion: {question} \n Table: {table}"
    res = llm.invoke(prompt)
    code = res.content.replace('```python','').replace('```','')
    return code

def exec_code(code, table):  
    try: 
        loc = locals()   
        if not "run()" in code:
            exec(code + f"\nr = run({table})\n", globals(), loc)
        else: 
            exec(code + "\nr = run()\n", globals(), loc)
        return loc['r']
    except Exception as e:
            s = '[Error]'+ str(e)
            print(s)
            return (s,'')
#table = "[['', '', 'Years Ended September 30,', ''], ['', '2019', '2018', '2017'], ['Fixed Price', '$  1,452.4', '$  1,146.2', '$  1,036.9'], ['Other', '44.1', '56.7', '70.8'], ['Total sales', '$1,496.5', '$1,202.9', '$1,107.7']]"
#question = 'What is the change in Fixed Price in 2019 from 2018?'
table = "[['', 'December 31,', ''], ['', '2019', '2018'], ['Trade accounts receivable, net, noncurrent (Note 2)', '$26,496', '$15,948'], ['Equity method investments (Note 1)', '9,254', '9,702'], ['Net deferred tax assets, noncurrent (Note 20)', '6,774', '5,797'], ['Rent and other deposits', '6,106', '5,687'], ['Value added tax receivables, net, noncurrent', '592', '519'], ['Other', '6,723', '5,711'], ['', '$55,945', '$43,364']]"
question = 'What was the percentage change in Value added tax receivables, net, noncurrent in 2019 from 2018?'
code = gen_code(llm, question, table)
exec_code(code, table)

(14.065510597302506, '%')

In [16]:
from tqdm.notebook import tqdm as log_progress

res = []
for i, item in devdf.iterrows():
    table = item['table']['table']
    #print (i)
    for q in item['questions']:        
        #if q['answer_type'] == 'arithmetic' and 'table' in q['answer_from'] : 
        if True: 
            code = gen_code(llm, q, table)
            r = exec_code(code, table)
            err =  None
            if r is None:
                (pred_value, pred_unit) = (None, None)
            else:                
                if len(r) == 2:
                    (pred_value, pred_unit) = r                    
                    #print(pred_value, type(pred_value))
                    if isinstance(pred_value, str) and  pred_value.startswith('[Error]'):
                        (pred_value, pred_unit) = (None, None)
                        err,_ = r                                                        
                        
                elif len(r) == 1:
                    pred_value = r                
                else:
                    (pred_value, pred_unit) = (None, None)        
                
            #res.append({"table":table, "q":q, "pred":pred, "code": code})
            res.append(({"answer_type":q["answer_type"], "answer": q["answer"], 'scale': q["scale"]}, pred_value, pred_unit, q, code, item['table'], err))


('166', '€m')


Exception ignored in: <function tqdm.__del__ at 0x7f9f98cceb90>
Traceback (most recent call last):
  File "/opt/conda/envs/arpad_tab/lib/python3.10/site-packages/tqdm/std.py", line 1148, in __del__
    self.close()
  File "/opt/conda/envs/arpad_tab/lib/python3.10/site-packages/tqdm/notebook.py", line 282, in close
    self.disp(bar_style='success', check_delay=False)
AttributeError: 'tqdm_notebook' object has no attribute 'disp'


KeyboardInterrupt: 

In [9]:
from tatqa_metric import TaTQAEmAndF1

metrics = TaTQAEmAndF1()

for ans, pred, pred_scale, _,_, _,_ in res:
    metrics(ans, pred, pred_scale)
pred_em, pred_f1, scale_score, op_score = metrics.get_overall_metric(reset=True)
print( pred_em, pred_f1, scale_score)

special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
0.5173860911270983 0.5421822541966426 0.5965227817745803


In [28]:
import pickle
with open('res_p8_unit.pickle', 'wb') as f:
    pickle.dump(res,f)

In [18]:
print([i for i in res if i[-1]  is not None][0])


({'answer_type': 'arithmetic', 'answer': 14.07, 'scale': 'percent'}, None, None, {'uid': 'a4f782f1-de3b-47a3-87af-1cb33c112b39', 'order': 6, 'question': 'What was the percentage change in Value added tax receivables, net, noncurrent in 2019 from 2018?', 'answer': 14.07, 'derivation': '(592-519)/519', 'answer_type': 'arithmetic', 'answer_from': 'table', 'rel_paragraphs': [], 'req_comparison': False, 'scale': 'percent'}, "def run():\n    table = [['', 'December 31,', ''], ['', '2019', '2018'], ['Trade accounts receivable, net, noncurrent (Note 2)', '$26,496', '$15,948'], ['Equity method investments (Note 1)', '9,254', '9,702'], ['Net deferred tax assets, noncurrent (Note 20)', '6,774', '5,797'], ['Rent and other deposits', '6,106', '5,687'], ['Value added tax receivables, net, noncurrent', '592', '519'], ['Other', '6,723', '5,711'], ['', '$55,945', '$43,364']]\n    \n    value_2019 = int(table[5][1])\n    value_2018 = int(table[5][2])\n    \n    percentage_change = ((value_2019 - value_2

In [20]:
len((1,2))

2

In [32]:
res[0][-1]

In [16]:
def run():
    table = [['', 'December 31,', ''], ['', '2019', '2018'], ['Trade accounts receivable, net, noncurrent (Note 2)', '$26,496', '$15,948'], ['Equity method investments (Note 1)', '9,254', '9,702'], ['Net deferred tax assets, noncurrent (Note 20)', '6,774', '5,797'], ['Rent and other deposits', '6,106', '5,687'], ['Value added tax receivables, net, noncurrent', '592', '519'], ['Other', '6,723', '5,711'], ['', '$55,945', '$43,364']]
    
    value_2019 = int(table[5][1])
    value_2018 = int(table[5][2])
    
    percentage_change = ((value_2019 - value_2018) / value_2018) * 100
    
    return (round(percentage_change, 2), 'percent')
run()

ValueError: invalid literal for int() with base 10: '6,106'